In [14]:
%matplotlib inline


import librosa
import pandas
import numpy
import sklearn
import sklearn.model_selection

import matplotlib.pyplot as plt

In [10]:
labels = pandas.read_csv('data/ff1010bird.labels.csv', index_col='itemid')
labels.head(3)

,datasetid,hasbird
itemid,,
64486,ff1010bird,0
2525,ff1010bird,0
44981,ff1010bird,0


In [13]:
len(labels), labels.hasbird.mean()

(7690, 0.2516254876462939)

In [33]:
pos = labels[labels.hasbird == 1].head(10)
neg = labels[labels.hasbird == 0].head(10)
v = neg.index.values
'{{{}}}'.format(','.join(str(i) for i in v))

'{64486,2525,44981,101323,165746,38232,104540,157473,132129,127302}'

In [ ]:
p = 'data/ff1010bird/{}.wav'.format(pos.index.values[0])
y, sr = librosa.load(p, offset=0)

mfcc = librosa.feature.mfcc(y=y, sr=sr)
mfcc.shape